<a href="https://colab.research.google.com/github/futurexskill/bigdata/blob/master/pyspark_bank_marketing_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install JDK
## Install Spark
## Set Environment variables
## Create a Spark Session

In [1]:
#Instalacja Sparka na colabie
!git clone https://github.com/djkormo/colab-examples.git

Cloning into 'colab-examples'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 121 (delta 44), reused 107 (delta 30), pack-reused 0
Receiving objects: 100% (121/121), 6.84 MiB | 4.62 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [2]:
!bash colab-examples/spark/install.bash

--2019-10-15 07:01:02--  http://apache.crihan.fr/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
Resolving apache.crihan.fr (apache.crihan.fr)... 195.221.21.36, 2001:660:7401:211::36
Connecting to apache.crihan.fr (apache.crihan.fr)|195.221.21.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230091034 (219M) [application/x-gzip]
Saving to: ‘spark-2.4.4-bin-hadoop2.7.tgz’

spark-2.4.4-bin-had 100%[===================>] 219.43M  11.4MB/s    in 22s     

2019-10-15 07:01:25 (10.2 MB/s) - ‘spark-2.4.4-bin-hadoop2.7.tgz’ saved [230091034/230091034]



In [0]:
# based on https://www.tutorialspoint.com/pyspark/pyspark_sparkcontext.htm

import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_ME'] = '/content/spark-2.4.4-bin-hadoop2.7'

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("First App") \
    .master("local[*]")\
    .getOrCreate()

In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

## Copy a data file to your local Colab environment

In [5]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/bank_prospects.csv

--2019-10-15 07:02:09--  https://raw.githubusercontent.com/futurexskill/bigdata/master/bank_prospects.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306 [text/plain]
Saving to: ‘bank_prospects.csv’

bank_prospects.csv  100%[===================>]     306  --.-KB/s    in 0s      

2019-10-15 07:02:10 (69.2 MB/s) - ‘bank_prospects.csv’ saved [306/306]



## Check if the file is copied

In [6]:
!ls


bank_prospects.csv  sample_data		       spark-2.4.4-bin-hadoop2.7.tgz
colab-examples	    spark-2.4.4-bin-hadoop2.7


# DataFrame

## Read the CSV file into a DataFrame

In [0]:
bankProspectsDF = spark.read.csv("bank_prospects.csv",header=True)

In [8]:
bankProspectsDF.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
|  27| 37000|Female|unknown|        N|
+----+------+------+-------+---------+



## Remove the record with unknow value in country column

In [0]:
bankProspectsDF1 = bankProspectsDF.filter(bankProspectsDF['country'] != "unknown")

In [10]:
bankProspectsDF1.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



##  Cast the String datatype to Integer/Float

In [11]:
bankProspectsDF1.printSchema()

root
 |-- Age: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



In [0]:
from pyspark.sql.types import IntegerType,FloatType

In [0]:
bankProspectsDF2 = bankProspectsDF1.withColumn("age", bankProspectsDF1["age"].cast(IntegerType())).withColumn("salary", bankProspectsDF1["salary"].cast(FloatType()))


In [14]:
bankProspectsDF2.printSchema()

root
 |-- age: integer (nullable = true)
 |-- salary: float (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



## Replace Age and Salary with average values of their respective column

import mean from sql.fuctions

In [0]:
from pyspark.sql.functions import mean

### Calculate "mean" value of the age

In [0]:
mean_age_val = bankProspectsDF2.select(mean(bankProspectsDF2['age'])).collect()

In [17]:
type(mean_age_val)

list

In [18]:
mean_age_val

[Row(avg(age)=22.11111111111111)]

In [0]:
mean_age = mean_age_val[0][0]

In [20]:
mean_age

22.11111111111111

### Calculate mean salary value

In [0]:
mean_salary_val = bankProspectsDF2.select(mean(bankProspectsDF2['salary'])).collect()

In [0]:
mean_salary = mean_salary_val[0][0]

In [23]:
mean_salary

31875.0

### Replace missing age with average value

In [24]:
bankProspectsDF2.show()

+----+-------+------+-------+---------+
| age| salary|Gender|Country|Purchased|
+----+-------+------+-------+---------+
|  18|20000.0|  Male|Germany|        N|
|  19|22000.0|Female| France|        N|
|  20|24000.0|Female|England|        N|
|  21|   null|  Male|England|        N|
|  22|50000.0|  Male| France|        Y|
|  23|35000.0|Female|England|        N|
|  24|   null|  Male|Germany|        N|
|  25|32000.0|Female| France|        Y|
|null|35000.0|  Male|Germany|        N|
|  27|37000.0|Female| France|        N|
+----+-------+------+-------+---------+



In [0]:
bankbankProspectsDF3 = bankProspectsDF2.na.fill(mean_age,["age"])

In [26]:
bankbankProspectsDF3.show()

+---+-------+------+-------+---------+
|age| salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|   null|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|   null|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000.0|  Male|Germany|        N|
| 27|37000.0|Female| France|        N|
+---+-------+------+-------+---------+



### Replace missing age with salary value

In [0]:
bankbankProspectsDF4 = bankbankProspectsDF3.na.fill(mean_salary,["salary"])

In [28]:
bankbankProspectsDF4.show()

+---+-------+------+-------+---------+
|age| salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|31875.0|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|31875.0|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000.0|  Male|Germany|        N|
| 27|37000.0|Female| France|        N|
+---+-------+------+-------+---------+



In [29]:
bankbankProspectsDF4.printSchema()

root
 |-- age: integer (nullable = true)
 |-- salary: float (nullable = false)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



## Write the transformed file to a new csv file 

In [0]:
bankbankProspectsDF4.write.format("csv").save("bank_prospects_transformed")

In [31]:
!ls

bank_prospects.csv	    colab-examples  spark-2.4.4-bin-hadoop2.7
bank_prospects_transformed  sample_data     spark-2.4.4-bin-hadoop2.7.tgz


In [32]:
!ls bank_prospects_transformed/

part-00000-cb7ebbd9-bad7-43a9-b538-cec67f065241-c000.csv  _SUCCESS


In [34]:
!cat bank_prospects_transformed/part-00000-cb7ebbd9-bad7-43a9-b538-cec67f065241-c000.csv

18,20000.0,Male,Germany,N
19,22000.0,Female,France,N
20,24000.0,Female,England,N
21,31875.0,Male,England,N
22,50000.0,Male,France,Y
23,35000.0,Female,England,N
24,31875.0,Male,Germany,N
25,32000.0,Female,France,Y
22,35000.0,Male,Germany,N
27,37000.0,Female,France,N
